## Differences between on fault and distributed seismicity explained by rock and fault rheology

Collettini, Barchi, De Paola, Trippetta and Tinti

In this notebook, we compute the b-values for "On-fault" seismicity, "Down-dip hangingwall seismicity" (DhWS), and for  three Clusters C1-C3 in the footwall of the Vettore fault, and reproduce related figures shown in Collettini et. al., (2022), under review. 
Let's get started!

In [ ]:
# import modules and setup 
# -------------------------------
import numpy as np
import datetime
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import matplotlib.patches as patches
from scipy.spatial.transform import Rotation as R

Computation of McLilliefors are from "Herrmann, M. and W. Marzocchi (2020). Mc-Lilliefors: A completeness magnitude that complies with the exponential-like Gutenberg-Richter relation. doiȷ 10.5281/zenodo.4162497."

In [ ]:
import sys
import logging
import pandas as pd
import plotly.io as pio

from mc_lilliefors import McLilliefors  # custom class
# -- Setup plotly

# Default plot type
# pio.renderers.default = 'plotly_mimetype+notebook'  # interactive
# pio.renderers.default = 'plotly_mimetype+notebook_connected'  # interactive with internet connection
pio.renderers.default = 'png'  # static

# Set plotly template
# pio.templates.default = 'plotly'  # [default]
pio.templates.default = 'simple_white'  # publication-friendly

# -- Setup logger
logger = logging.getLogger()
c_handler = logging.StreamHandler(sys.stdout)  # avoid red colored cells by not logging to stderr
logger.setLevel(logging.INFO)
c_handler.setFormatter(logging.Formatter('%(message)s'))
logger.handlers = [c_handler]

In [ ]:
#from statsmodels.distributions.empirical_distribution import ECDF
# pip install dc_stat_think
import dc_stat_think as dcst

In [ ]:
def compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=None):
    """Compute the b-value and its confidence interval."""
    
## This function returns the 1/b value given the selected catalogue and the minimum magnitude 
    
    m = mags[mags >= mt]

    # Compute b-value: b
    b = ((np.mean(m) - (mt-interval/2)) * np.log(10) )
    
    # Draw bootstrap replicates
    if n_reps is None:
        return b
    else:
        m_bs_reps = dcst.draw_bs_reps(m, np.mean, size=n_reps)

        # Compute b-value from replicates: b_bs_reps
        b_bs_reps = ((m_bs_reps - (mt-interval/2)) * np.log(10))
        
        # Compute confidence interval: conf_int
        conf_int = np.percentile(b_bs_reps, perc)
    
        return b, conf_int


In [ ]:
import pyproj
utm33 = pyproj.Proj(proj='utm',zone=33, ellps='WGS84', preserve_units=False)
#utm33 = pyproj.Proj("+proj=utm +zone=33, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [ ]:
def distance_point_from_plane(x, y, z, normal, origin):
    d = -normal[0]*origin[0]-normal[1]*origin[1]-normal[2]*origin[2]
    dist = np.abs(normal[0]*x+normal[1]*y+normal[2]*z+d)
    dist = dist/np.sqrt(normal[0]**2+normal[1]**2+normal[2]**2)
    return dist

### Download the Catalog 

The seismicity catalogue used in this work is published in Tan et al., 2021 and is available at the Zenodo dataset repository (doi 10.5281/zenodo.4662870).

In [ ]:
# to run this cell donwload the Tan catalog, put in the same folder of this notebook and rename as Amatrice_Tanetal2021 
am=pd.read_csv('Amatrice_Tanetal2021', delimiter='\s+', header= 18)
am.columns = [ "year","month","day","hour","min","secmsec","lat", "lon", "depth","EH1","EH2","AZ","EZ","Ml","Mw","Id"]
am =am.drop(['EH1','EH2','AZ','EZ'], axis = 1)

In [ ]:
am

In [ ]:
df = pd.DataFrame({'year': am['year'],
                       'month': am['month'],
                       'day': am['day'],'hour': am['hour'],
                       'minute': am['min'], 'second':am['secmsec']})
time=pd.to_datetime(df, format = '%d/%m/%y %H:%M:%S')
#Convert argument to datetime
am['time']=time


In [ ]:
am=am.drop(['year', 'month','day','hour','min','secmsec'], axis=1)

In [ ]:
#Set the interval of the available Mw values (take it info from the catalogue)
interval=0.01

In [ ]:
%matplotlib inline
fig=plt.figure (figsize=(15,5))
plt.plot(am['time'],am['Mw'],'o')
plt.ylabel('Mw',fontsize=24)
plt.xlabel('time',fontsize=24)

In [ ]:
# to utm33
utmx_am,utmy_am=utm33(np.array(am['lon']),np.array(am['lat']))

In [ ]:
largeeve=am[(am['Mw'] >= 5.)]
largeeve= largeeve.reset_index(drop=True)
largeeve

In [ ]:
norcia_ipo=am[(am['Mw'] >= 6.)&(am['time'] >= '2016-10-30')]
norcia_ipo

In [ ]:
#Reference earthquake that is the center of the section Norcia 30 10 2016
lon_ref=float(norcia_ipo['lon'])
lat_ref=float(norcia_ipo['lat'])
depth_ref=float(norcia_ipo['depth'] )

utmx_ref,utmy_ref=utm33(lon_ref,lat_ref)
mainev_ref=[utmx_ref/1000,utmy_ref/1000, -depth_ref ]
mainev_ref=np.array(mainev_ref)

### Mappa all dataset

In [ ]:
%matplotlib inline
whe_largeeve=np.where(am['Mw'] >= 1.5)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(utmx_am[whe_largeeve]/1000,utmy_am[whe_largeeve]/1000, s=0.05,marker='o', c='#000080', alpha=.5, label='Tan et al., 2021')
ax.scatter(mainev_ref[0], mainev_ref[1],s=100, c='r',marker='*', label='Norcia Mainshock')


plt.legend(fontsize=11, markerscale=2)
plt.xlabel('x (km)',fontsize=18)
plt.ylabel('y (km)',fontsize=18)
ax=plt.gca()
ax.set_aspect('equal')
ax.grid()

## Dataset whole sequence without STAI

The figure reported below is Figure 6 of the supplementary material of Herrmann et al., (Research Square, https://orcid.org/0000-0002-2342-1970). According to the Authors, this plot is useful to define the necessary “safety margin” to reduce the bias due to short-term aftershock incompleteness (STAI). According to this analysis, the Norcia mainshock has the strongest influence on STAI, and +2days have to be removed to avoid the influence of STAI on the b-values. For the other mainshocks the influence is limited at +0.8, +0.6 and +0.4 for Amatrice, Visso and Campotosto mainshocks, respectively.

<img src="WarnerSTAI.png" >

To compute the b-value for the seismicity "on-fault" and "DhWS" we have to select the period.In the paper we show the inferred b-value for the time period before Visso-Norcia and for the whole dataset. Please select PERIOD=0 for the whole dataset and PERIOD=1 for the seismicity before Visso-Norcia.

In [ ]:
#SELECT PERIOD:
#BEFORE VISSO-NORCIA =1
#WHOLE PERIOD =0
PERIOD=0

In [ ]:
#EVENTS FOR THE WHOLE PERIOD OR BEFORE NORCIA ACCOUNTING FOR STAI from Herrmann et al, under review
start_date0 = '2016-08-15 00:01 '
end_date0 = '2016-08-24 01:36:34'#'2016-12-09 06:39' STAI 19,2 ore
start_date1 = '2016-08-24 20:36 '
end_date1 = '2016-10-26 19:18:10 '
start_date2 = '2016-10-27 09:18:10 ' #STAI 14,4 hours
end_date2 = '2016-10-30 06:40:19'
start_date3 = '2016-11-01 06:40 ' #STAI 48 hours
end_date3 = '2017-01-18 09:26'
start_date4 = '2017-01-18 21:00 ' # 9,6 hours
end_date4 = '2017-08-15 22:16'

if PERIOD==0:
    am_poM1 = am[((am['time'] >= start_date0) & (am["time"] <= end_date0)) |((am['time'] >= start_date1) & (am["time"] <= end_date1)) | ((am['time'] >= start_date2) & (am["time"] <= end_date2)) | ((am['time'] >= start_date3) & (am["time"] <= end_date3)) | ((am['time'] >= start_date4) & (am["time"] <= end_date4))]
    am_poM1= am_poM1.reset_index(drop=True)
elif PERIOD==1:
    am_poM1 = am[((am['time'] >= start_date0) & (am["time"] <= end_date0)) |((am['time'] >= start_date1) & (am["time"] <= end_date1)) ]
    am_poM1= am_poM1.reset_index(drop=True)
am_poM1

In [ ]:
am_poM1=am_poM1[(am_poM1['depth'] <= 50.0)]
am_poM1= am_poM1.reset_index(drop=True)
am_poM1

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(am_poM1['time'],am_poM1['Mw'],'o')

In [ ]:
#Define the vector normal to the plane where we want to plot the seismicity
#strike_ref=155
#section normal_tostrike=strike_ref-90   
dip_angle=90
normal_tostrike=155-90  #direction of the section to plot
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]

In [ ]:
#convert the geographical coordinates in utm.
utmx_am_poM1,utmy_am_poM1=utm33(np.array(am_poM1['lon']),np.array(am_poM1['lat']))


In [ ]:

#compute the distance from the selected plane
distam_ref=distance_point_from_plane(utmx_am_poM1/1000,utmy_am_poM1/1000, -am_poM1['depth'], normal_ref, mainev_ref)

#set the distance threshold from the plane 
x_along_strike=1 # km
resultdist1m = np.where(distam_ref <x_along_strike)

#X location along the section
X_onsection1m=+(utmy_am_poM1[resultdist1m]/1000-mainev_ref[1])*normal_ref_vertical[0]-(utmx_am_poM1[resultdist1m]/1000-mainev_ref[0])*normal_ref_vertical[1]

# Mainshock or selected event on section
fm_onsection_ref=+(utmy_ref/1000-mainev_ref[1])*normal_ref_vertical[0]-(utmx_ref/1000-mainev_ref[0])*normal_ref_vertical[1]

In [ ]:
%matplotlib inline
#%matplotlib inline
#project the depth along dip
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))

ax.scatter(X_onsection1m,(-am_poM1['depth'].loc[resultdist1m])/np.sin(dip_angle*np.pi/180),c='k',s=10,marker='.',alpha=.2,label='x')
ax.scatter(fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180), marker='*', c='r',s=250, label='Mw 6.5')

ax.set_ylim(-12,0)
ax.set_xlim(-10,20)
ax.set_title('distance along strike (km) +/- '+str(x_along_strike),fontsize=18)
ax.set_aspect('equal', adjustable='box')
ax.set_ylabel('along depth')
ax.set_xlabel('along strike')
fig.savefig("./figure/seismicity_section.pdf", bbox_inches='tight')

In [ ]:
#catalogue to use for the computation of M_lilliefors and b-value
df = am_poM1.copy()


In [ ]:
## Herrmann, M. and W. Marzocchi (2020). Mc-Lilliefors: A completeness magnitude that complies with the
## exponential-like Gutenberg-Richter relation. doiȷ 10.5281/zenodo.4162497.

# Reload class (e.g., to consider modifications in the code)
import importlib
import mc_lilliefors
importlib.reload(mc_lilliefors)
from mc_lilliefors import McLilliefors

lill = McLilliefors(
    df.Mw,
    # signif_lev=0.1  # [default: 0.1]
)

lill.calc_testdistr_mcutoff(
    n_repeats=50,  # number of iterations for the random noise
    Mstart=1.0,  # lowest magnitude for which to perform the test
    # log=False,  # whether to show anythe progress bar
)



In [ ]:
## Herrmann, M. and W. Marzocchi (2020). Mc-Lilliefors: A completeness magnitude that complies with the
## exponential-like Gutenberg-Richter relation. doiȷ 10.5281/zenodo.4162497.

fig = lill.plot_testdist_expon_mcutoff()
fig.layout.update(
    xaxis_range=(1, 6.0),
)

fig.show(width=fig.layout.width, height=fig.layout.height)

In [ ]:
Mc = lill.estimate_Mc_expon_test()

print("Mc-Lilliefors: %s\n   --> number of events ≥ Mc-Lilliefors: %d" % (Mc, lill.estimates['n_compl']))

In [ ]:

Mc_all=Mc

In [ ]:
catalogue=am_poM1.copy()

In [ ]:

# Magnitude bins
min_mag=np.min(catalogue['Mw']) # estremi per il plot
max_mag=np.max(catalogue['Mw'])+interval #estremi per il plot
min_mag_bin=Mc
max_mag_bin=np.max(catalogue['Mw'])+interval

bins_all = np.arange(min_mag_bin, max_mag_bin, interval)
# Magnitude bins for plotting - we will re-arrange bins later
plot_bins_all = np.arange(min_mag, max_mag, interval)

###########################################################################
# Generate distribution
###########################################################################
# Generate histogram
hist = np.histogram(catalogue['Mw'],bins=bins_all)
hist_plot = np.histogram(catalogue['Mw'],bins=plot_bins_all)

# Reverse array order
hist = hist[0][::-1]
bins_all = bins_all[::-1]
#plot 
hist_plot = hist_plot[0][::-1]
plot_bins_all = plot_bins_all[::-1]

# Calculate cumulative sum
cum_hist = hist.cumsum()
# Ensure bins have the same length has the cumulative histogram.
# Remove the upper bound for the highest interval.
bins_all = bins_all[1:]


cum_hist_plot_all = hist_plot.cumsum()
plot_bins_all = plot_bins_all[1:]
wherecum=np.where(cum_hist>0)
log_cum_sum = np.log10(cum_hist[wherecum])

In [ ]:
%matplotlib inline
# Compute b-value and confidence interval
#b, conf_int = b_value(mags, mt, perc=[2.5, 97.5], n_reps=10000)
mags=catalogue['Mw']
mt=Mc
compute_b_value_mle, conf_int = compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=10000)
# Generate samples to for theoretical ECDF
m_theor = np.random.exponential(compute_b_value_mle/np.log(10), size=100000) + mt 
# Plot the theoretical CDF
_ = plt.plot(*dcst.ecdf(m_theor))

# Plot the ECDF (slicing mags >= mt)
_ = plt.plot(*dcst.ecdf(mags[mags >= mt]), marker='.', linestyle='none')

# Pretty up and show the plot
_ = plt.xlabel('magnitude')
_ = plt.ylabel('ECDF')
#_ = plt.xlim(2.8, 6.2)
plt.show()

# Report the results
print("""
b-value: {0:.5f}
95% conf int: [{1:.5f}, {2:.5f}]""".format(1/compute_b_value_mle, *conf_int**(-1)))
compute_b_value_mle=1/compute_b_value_mle

In [ ]:

b_all=compute_b_value_mle

In [ ]:
num_eq=len(catalogue['Mw'])
## Get annual rate
cum_annual_rate = cum_hist

new_cum_annual_rate_all = []
for i in cum_annual_rate:
    new_cum_annual_rate_all.append(i+1e-20)

In [ ]:
# Generate data to plot maximum likelihood linear curve

num_eve=len(catalogue['Mw'].loc[np.where(catalogue['Mw']>min_mag_bin)])
mle_fit = -1.0 * compute_b_value_mle * bins_all + 1.0 * compute_b_value_mle * min_mag_bin + np.log10(num_eve)
log_mle_fit = []
for value in mle_fit:
    log_mle_fit.append(np.power(10,value))
    
# Compare b-value of 1
fit_data = -1.0 * bins_all + min_mag_bin + np.log10(num_eve)
log_fit_data = []
for value in fit_data:
    log_fit_data.append(np.power(10,value))

print('compute_b_value_mle=',compute_b_value_mle)

In [ ]:
b_all=compute_b_value_mle
log_mle_fit_all=log_mle_fit.copy()

In [ ]:
# Plotting
# Plotting
wherecumplot_all=np.where(cum_hist_plot_all>0)
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(plot_bins_all[wherecumplot_all],cum_hist_plot_all[wherecumplot_all],'xr',label = 'All events of selected period')
ax.scatter(bins_all, new_cum_annual_rate_all, label = 'Events with M>M_lilliefors')

ax.plot(bins_all, log_mle_fit, c = 'b', label = 'bvalue_MLE= %f'%b_all)
ax.plot(bins_all, log_fit_data, c = 'k',linestyle='--', label = 'b=1')

#ax.plot(bins, ls_fit2, c = 'k')
ax.set_yscale('log')
ax.legend(fontsize=24)
#ax.set_ylim([min(new_cum_annual_rate) * 0.1, max(new_cum_annual_rate) * 10.])
ax.set_ylim(1e-2, max(new_cum_annual_rate_all) * 100.)
ax.set_xlim([min_mag - 2.5, max_mag + 0.5])
ax.set_ylabel('Number events', fontsize=20)
ax.set_xlabel('Magnitude',fontsize=20)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
plt.show()


# On Fault

In [ ]:
#strike_ref=150
#normal_tostrike=strike_ref-90  
dip_angle=40
normal_tostrike=155   #direction of the section to plot
#normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]


We use the Rotation module to retrieve the normal vector to the selected plane

In [ ]:
#dip_angle=90-angle_dip
#strike_angle=-angle_strike+90
dipangle=90-40
strikeangle=-155+90


#vx1 = R.from_euler('x', 0, degrees=True) #first rotation around X; for vertical section  dip is ZERO 
vx1 = R.from_euler('x', dipangle, degrees=True) #first rotation around X; for section with dip 40 we have to use 90-dip 
vx2 = R.from_euler('z', strikeangle, degrees=True) #rotation around Z for strike


In [ ]:
dippe=np.array([0,0,-1])
dippe_rotated2=vx1.apply(dippe)
dippe_rotated1=vx2.apply(dippe_rotated2)


In [ ]:
strikke=np.array([1,0,0])
strikke_rotated2=vx1.apply((strikke))
strikke_rotated1=vx2.apply((strikke_rotated2))


In [ ]:

ppx=strikke_rotated1
ppy=dippe_rotated1


In [ ]:
#UTM33
utmx_am_poM1,utmy_am_poM1=utm33(np.array(am_poM1['lon']),np.array(am_poM1['lat']))   # 1 day pre M 6.5

#distance reference point
distpoM1_ref=distance_point_from_plane(utmx_am_poM1/1000,utmy_am_poM1/1000, -am_poM1['depth'], normal_ref, mainev_ref)

# selection distance < x
resultdist1poM1 = np.where(distpoM1_ref <0.5)

vettore_dist=np.array([(utmx_am_poM1[resultdist1poM1]/1000-mainev_ref[0]),(utmy_am_poM1[resultdist1poM1]/1000-mainev_ref[1]),(+am_poM1['depth'].loc[resultdist1poM1]+mainev_ref[2])])

tot_dist=np.array(np.sqrt((utmx_am_poM1[resultdist1poM1]/1000-mainev_ref[0])**2+(utmy_am_poM1[resultdist1poM1]/1000-mainev_ref[1])**2+(-am_poM1['depth'].loc[resultdist1poM1]-mainev_ref[2])**2))

dist_project=np.array((np.sqrt(tot_dist**2-(distpoM1_ref.loc[resultdist1poM1])**2)))


In [ ]:
resultdist1poM1

In [ ]:
# X and Y location on the selected section plane  
X_onsection1poM1=np.empty([0])
Y_onsection1poM1=np.empty([0])
for i in range(len(vettore_dist[1,:])):
    rec_rotX=np.dot(ppx,vettore_dist[:,i])
    rec_rotY=np.dot(ppy,vettore_dist[:,i])
    #print(rec_rot.shape)
    #print(rec_rot)
    X_onsection1poM1=np.append(X_onsection1poM1, [rec_rotX], axis=0)
    Y_onsection1poM1=np.append(Y_onsection1poM1, [rec_rotY], axis=0)

In [ ]:
%matplotlib inline
dip_angle=40
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))

##proiezione along distance - perpendicular to the fault plane. 
ax.scatter(X_onsection1poM1,Y_onsection1poM1-depth_ref/np.sin(dip_angle*np.pi/180),c='b',s=0.2,marker='.',alpha=.9,label='x')
ax.scatter(0, 0-depth_ref/np.sin(dip_angle*np.pi/180), marker='*', c='r',s=250, label='Mw 6.5')

ax.set_ylabel('along dip',fontsize=20)
ax.set_xlabel('along strike',fontsize=20)
ax.set_ylim(-15,0)
ax.set_xlim(-20,40)
ax.set_aspect('equal', adjustable='box')


In [ ]:
#create catalog with events having a distance < 0.5 km from the fault plane 
onfault_eve=am_poM1.copy()
onfault_eve=onfault_eve.loc[resultdist1poM1]
onfault_eve= onfault_eve.reset_index(drop=True)

In [ ]:
onfault_eve

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(onfault_eve['time'],onfault_eve['Mw'],'o')

In [ ]:
utmx_onf,utmy_onf=utm33(np.array(onfault_eve['lon']),np.array(onfault_eve['lat']))   # 1 day pre M 6.5

In [ ]:
dip_angle=40
#where_onf = (X_onsection_onf> -10) &(X_onsection_onf< 10)&(onfault_eve['depth']<np.abs(mainev_ref[2]))&(onfault_eve['depth']>+2)
#where_onf = (X_onsection_onf> -5) &(X_onsection_onf< 5)&(onfault_eve['depth']<np.abs(mainev_ref[2]-0.0))&(onfault_eve['depth']>+2.)
where_onf = (X_onsection1poM1> -5.) &(X_onsection1poM1< 5.)&((Y_onsection1poM1-depth_ref/np.sin(dip_angle*np.pi/180))>=-(+depth_ref)/np.sin(dip_angle*np.pi/180))&((Y_onsection1poM1-depth_ref/np.sin(dip_angle*np.pi/180))<-2.2/np.sin(40*np.pi/180))


In [ ]:
onfault_eve[where_onf]

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
onfault_eve['Mw'][where_onf].plot.hist(ax=ax, bins=50)#, bottom=1)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(0, 313)
ax.scatter(utmx_onf[where_onf]/1000, utmy_onf[where_onf]/1000,-onfault_eve['depth'].loc[where_onf], c='k',marker='.')
ax.scatter(utmx_ref/1000,utmy_ref/1000,-depth_ref,marker='*',c='r',s=100)

In [ ]:
close_on=onfault_eve[where_onf]
close_on= close_on.reset_index(drop=True)
close_on



In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(close_on['time'],close_on['Mw'],'o')

In [ ]:
utmx_close,utmy_close=utm33(np.array(close_on['lon']),np.array(close_on['lat']))   # 1 day pre M 6.5
X_onsection_close_on=+(utmy_close/1000-mainev_ref[1])*normal_ref_vertical[0]-(utmx_close/1000-mainev_ref[0])*normal_ref_vertical[1]

In [ ]:
%matplotlib inline
fig=plt.figure(figsize=(14,5))
plt.plot(am['time'],am['Mw'],'kx')
plt.plot(onfault_eve['time'],onfault_eve['Mw'],'o')
plt.plot(close_on['time'],close_on['Mw'],'ro')

In [ ]:
#strike_ref=150
#normal_tostrike=strike_ref-90   
dip_angle=90
normal_tostrike=155-90   #direction of the section to plot
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]

In [ ]:
#distance reference point
dist_ref=distance_point_from_plane(utmx_close/1000,utmy_close/1000, -close_on['depth'], normal_ref, mainev_ref)

# X on cross section 
X_on=+(utmy_close/1000-mainev_ref[1])*normal_ref_vertical[0]-(utmx_close/1000-mainev_ref[0])*normal_ref_vertical[1]


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 7))
ax.scatter(X_onsection1m,(-am_poM1['depth'].loc[resultdist1m])/np.sin(dip_angle*np.pi/180),c='k',s=1,marker='.',alpha=.2,label='x')

#
ax.scatter(X_on,-close_on['depth']/np.sin(dip_angle*np.pi/180),c='b',s=1,marker='.',alpha=0.2)
ax.scatter(fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180), marker='*', c='c',s=250, label='Mw 6.5')

ax.set_ylim(-10,0)
ax.set_xlim(-5,15)
ax.set_aspect('equal', adjustable='box')

In [ ]:
#catalogue to use
df = close_on.copy()
df

In [ ]:
# Reload class (e.g., to consider modifications in the code)
import importlib
import mc_lilliefors
importlib.reload(mc_lilliefors)
from mc_lilliefors import McLilliefors

lill = McLilliefors(
    df.Mw,
    # signif_lev=0.1  # [default: 0.1]
)

lill.calc_testdistr_mcutoff(
    n_repeats=50,  # number of iterations for the random noise
    Mstart=1.0,  # lowest magnitude for which to perform the test
    # log=False,  # whether to show anythe progress bar
)



In [ ]:
fig = lill.plot_testdist_expon_mcutoff()
fig.layout.update(
    xaxis_range=(1, 6.0),
)

fig.show(width=fig.layout.width, height=fig.layout.height)

In [ ]:
Mc = lill.estimate_Mc_expon_test()

print("Mc-Lilliefors: %s\n   --> number of events ≥ Mc-Lilliefors: %d" % (Mc, lill.estimates['n_compl']))

In [ ]:
Mc_on=Mc

In [ ]:
catalogue=close_on.copy()
magnitudes=catalogue['Mw']


In [ ]:
# Magnitude bins

min_mag=np.min(catalogue['Mw']) # estremi per il plot
max_mag=np.max(catalogue['Mw'])+interval #estremi per il plot
min_mag_bin=Mc
max_mag_bin=np.max(catalogue['Mw'])+interval

bins = np.arange(min_mag_bin, max_mag_bin, interval)
# Magnitude bins for plotting - we will re-arrange bins later
plot_bins = np.arange(min_mag, max_mag, interval)

wwww=np.where(catalogue['Mw']>min_mag_bin)
plt.hist(catalogue['Mw'].loc[wwww], bins, alpha = 0.5, label='a')
plt.hist(catalogue['Mw'], plot_bins, alpha = 0.5, label='b')
###########################################################################
# Generate distribution
###########################################################################
# Generate histogram
hist = np.histogram(catalogue['Mw'],bins=bins)
hist_plot = np.histogram(catalogue['Mw'],bins=plot_bins)

# Reverse array order
hist = hist[0][::-1]
bins = bins[::-1]
#plot 
hist_plot = hist_plot[0][::-1]
plot_bins = plot_bins[::-1]

# Calculate cumulative sum
cum_hist = hist.cumsum()
# Ensure bins have the same length has the cumulative histogram.
# Remove the upper bound for the highest interval.
bins = bins[1:]


cum_hist_plot = hist_plot.cumsum()
plot_bins = plot_bins[1:]
wherecum=np.where(cum_hist>0)
#log_cum_sum = np.log10(cum_hist[2::])
log_cum_sum = np.log10(cum_hist[wherecum])


In [ ]:
%matplotlib inline
mags=catalogue['Mw']
mt=Mc
compute_b_value_mle, conf_int = compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=10000)
# Generate samples to for theoretical ECDF
m_theor = np.random.exponential(compute_b_value_mle/np.log(10), size=100000) + mt 
# Plot the theoretical CDF
_ = plt.plot(*dcst.ecdf(m_theor))

# Plot the ECDF (slicing mags >= mt)
_ = plt.plot(*dcst.ecdf(mags[mags >= mt]), marker='.', linestyle='none')

# Pretty up and show the plot
_ = plt.xlabel('magnitude')
_ = plt.ylabel('ECDF')
#_ = plt.xlim(2.8, 6.2)
plt.show()

# Report the results
print("""
b-value: {0:.5f}
95% conf int: [{1:.5f}, {2:.5f}]""".format(1/compute_b_value_mle, *conf_int**(-1)))
compute_b_value_mle=1/compute_b_value_mle






In [ ]:
# b- value for the on-fault seismicity
b_on=compute_b_value_mle

In [ ]:
num_eq=len(magnitudes)
annual_num_eq = num_eq
## Get annual rate
cum_annual_rate = cum_hist

new_cum_annual_rate = []
for i in cum_annual_rate:
    new_cum_annual_rate.append(i+1e-20)

In [ ]:
   
# Generate data to plot maximum likelihood linear curve

num_eve=len(catalogue['Mw'].loc[np.where(catalogue['Mw']>min_mag_bin)])
mle_fit = -1.0 * compute_b_value_mle * bins + 1.0 * compute_b_value_mle * min_mag_bin + np.log10(num_eve)
log_mle_fit = []
for value in mle_fit:
    log_mle_fit.append(np.power(10,value))
    
# Compare b-value of 1
fit_data = -1.0 * bins + min_mag_bin + np.log10(num_eve)
log_fit_data = []
for value in fit_data:
    log_fit_data.append(np.power(10,value))

print(log_cum_sum[-1],np.log10(num_eve))
print('compute_b_value_mle=',compute_b_value_mle)

In [ ]:
bin_on=bins.copy()
log_mle_fit_on=log_mle_fit.copy()

In [ ]:
wherecumplot=np.where(cum_hist_plot>0)
%matplotlib inline
# Plotting
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(plot_bins[wherecumplot],cum_hist_plot[wherecumplot],'xr',label = 'On-fault events of selected period')
ax.scatter(bins, new_cum_annual_rate, label = 'On-fault events with M>M_lilliefors')

ax.plot(bins, log_mle_fit, c = 'b', label = 'bvalue_MLE= %.2f'%b_on)
ax.plot(bins, log_fit_data, c = 'k',linestyle='--', label = 'b=1')



#ax.plot(bins, ls_fit2, c = 'k')
ax.set_yscale('log')
ax.legend(fontsize=24)
#ax.set_ylim([min(new_cum_annual_rate) * 0.1, max(new_cum_annual_rate) * 10.])
ax.set_ylim(1e-2, max(new_cum_annual_rate) * 100.)
ax.set_xlim([min_mag - 1, max_mag + 0.5])
ax.set_ylabel('Number events',fontsize=20)
ax.set_xlabel('Magnitude',fontsize=20)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
plt.show()




## DHwS (distributed down-dip hangingwall seismicity)

In [ ]:

#Reference earthquake that is the center of the section Norcia 30 10 2016 abbassato a 8.5 depth
lon_ref=float(norcia_ipo['lon'])
lat_ref=float(norcia_ipo['lat'])
depth_ref_off=8.4 #era 8.3 con 20 gradi  #era 8.7 con selezione collettini e dip 30

utmx_refoff,utmy_refoff=utm33(lon_ref,lat_ref)
mainev_ref_off=[utmx_refoff/1000,utmy_refoff/1000, -depth_ref_off ]
mainev_ref_off=np.array(mainev_ref_off)
#print(mainev_ref)

In [ ]:
#strike_ref=155
#normal_tostrike=strike_ref-90  
dip_angle=20# 30 selezione collettini 
normal_tostrike=155+180   #direction of the section to plot
#normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]


In [ ]:

#dip_angle=90-angle_dip
#strike_angle=-angle_strike+90
dipangle=90-160 # dip of 20 degrees toward east
strikeangle=-155+90

ddx1 = R.from_euler('x', dipangle, degrees=True) #first rotation around X; for section with dip 20 we have to use -90+dip 
ddx2 = R.from_euler('z', strikeangle, degrees=True) #rotation around Z for strike



In [ ]:
dippedd=np.array([0,0,-1])
dippedd_rotated2=ddx1.apply(dippedd)
dippedd_rotated1=ddx2.apply(dippedd_rotated2)

In [ ]:
strikkedd=np.array([1,0,0])
strikkedd_rotated2=ddx1.apply((strikkedd))
strikkedd_rotated1=ddx2.apply((strikkedd_rotated2))



In [ ]:

ppx=strikkedd_rotated1
ppy=dippedd_rotated1


In [ ]:
#distance reference point
distpoM1_refOFF=distance_point_from_plane(utmx_am_poM1/1000,utmy_am_poM1/1000, -am_poM1['depth'], normal_ref, mainev_ref_off)

# selection distance < x
resultdist1poM1OFF = np.where(distpoM1_refOFF <2. )# 2.2 con scelta collettini 30 gradi

vettore_distOFF=np.array([(utmx_am_poM1[resultdist1poM1OFF]/1000-mainev_ref_off[0]),(utmy_am_poM1[resultdist1poM1OFF]/1000-mainev_ref_off[1]),(+am_poM1['depth'].loc[resultdist1poM1OFF]+mainev_ref_off[2])])

tot_distOFF=np.array(np.sqrt((utmx_am_poM1[resultdist1poM1OFF]/1000-mainev_ref_off[0])**2+(utmy_am_poM1[resultdist1poM1OFF]/1000-mainev_ref_off[1])**2+(-am_poM1['depth'].loc[resultdist1poM1OFF]-mainev_ref_off[2])**2))

dist_projectOFF=np.array((np.sqrt(tot_distOFF**2-(distpoM1_refOFF.loc[resultdist1poM1OFF])**2)))



In [ ]:
X_onsection1poM1OFF=np.empty([0])
Y_onsection1poM1OFF=np.empty([0])
for i in range(len(vettore_distOFF[1,:])):
    rec_rotX=np.dot(ppx,vettore_distOFF[:,i])
    rec_rotY=np.dot(ppy,vettore_distOFF[:,i])
    #print(rec_rot.shape)
    #print(rec_rot)
    X_onsection1poM1OFF=np.append(X_onsection1poM1OFF, [rec_rotX], axis=0)
    Y_onsection1poM1OFF=np.append(Y_onsection1poM1OFF, [rec_rotY], axis=0)

In [ ]:
%matplotlib inline
print(dip_angle)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))

#-mainev_ref_off[2]/np.sin(dip_angle*np.pi/180)
ax.scatter(X_onsection1poM1OFF,Y_onsection1poM1OFF,c='b',s=0.2,marker='.',alpha=.9,label='x')
##proiezione along distance - perpendicular
ax.scatter(0, 0, marker='*', c='r',s=250, label='Mw 6.5')
#
#rect = patches.Rectangle((fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180)), 6, 5, angle=90,linewidth=2, edgecolor='r', facecolor='none')
#
## Add the patch to the Axes
#ax.add_patch(rect)
#rect1 = patches.Rectangle((fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180)), 6, -5, angle=90,linewidth=2, edgecolor='r', facecolor='none')
#
## Add the patch to the Axes
#ax.add_patch(rect1)

ax.set_ylabel('along dip',fontsize=20)
ax.set_xlabel('along strike',fontsize=20)
ax.set_ylim(-20,20)
ax.set_xlim(-20,20)
ax.set_aspect('equal', adjustable='box')


In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(am_poM1['lon'],am_poM1['lat'], c='gray',marker='.',alpha=0.5)
ax.scatter(am_poM1['lon'].loc[resultdist1poM1OFF],am_poM1['lat'].loc[resultdist1poM1OFF], c='r',marker='.')


In [ ]:
%matplotlib inline 
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(utmx_am_poM1/1000,utmy_am_poM1/1000, c='gray',marker='.',alpha=0.5)
ax.scatter(utmx_am_poM1[resultdist1poM1OFF]/1000,utmy_am_poM1[resultdist1poM1OFF]/1000, c='r',marker='.')


In [ ]:
#create catalog with events belong to DHWS

offfault_eve=am_poM1.copy()
offfault_eve=offfault_eve.loc[resultdist1poM1OFF]
offfault_eve= offfault_eve.reset_index(drop=True)


In [ ]:
utmx_off,utmy_off=utm33(np.array(offfault_eve['lon']),np.array(offfault_eve['lat']))   # 1 day pre M 6.5

In [ ]:
#strike_ref=150
#normal_tostrike=strike_ref-90   
dip_angle=90
normal_tostrike=155-90   #direction of the section to plot
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]

In [ ]:
##distance reference point
dist_orthog=distance_point_from_plane(utmx_off/1000,utmy_off/1000, -offfault_eve['depth'], normal_ref, mainev_ref_off)
#plt.plot(distpoM1_ref)
## selection distance < x
resultdist1_othog = np.where(dist_orthog <5)
#
# X on cross section 
X_onsec_othog_off=+(utmy_off/1000-mainev_ref_off[1])*normal_ref_vertical[0]-(utmx_off/1000-mainev_ref_off[0])*normal_ref_vertical[1]


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.scatter(X_onsec_othog_off[resultdist1_othog],-offfault_eve['depth'].loc[resultdist1_othog]/np.sin(dip_angle*np.pi/180),c='b',s=0.5,marker='.',alpha=.5,label='x')
ax.scatter(fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180), marker='*', c='r',s=250, label='Mw 6.5')
ax.scatter(X_on,-close_on['depth']/np.sin(dip_angle*np.pi/180),c='k',s=1,marker='.',alpha=0.2)
ax.set_title('5 km thick')
ax.set_ylim(-15,0)
ax.set_xlim(-20,20)
ax.set_aspect('equal', adjustable='box')

In [ ]:
dip_angle=20

In [ ]:
where_off = (X_onsection1poM1OFF> -5.) &(X_onsection1poM1OFF< 5.)&(Y_onsection1poM1OFF>1)&(Y_onsection1poM1OFF<8.)&(offfault_eve['depth']>5)&(offfault_eve['depth']<9.0)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
offfault_eve['Mw'][where_off].plot.hist(ax=ax, bins=50)#, bottom=1)

In [ ]:
close_off=offfault_eve[where_off]
close_off= close_off.reset_index(drop=True)
close_off

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(0, 313)
ax.scatter(utmx_off[where_off]/1000, utmy_off[where_off]/1000,-offfault_eve['depth'].loc[where_off], c='k',marker='.')


In [ ]:

dip_angle=90
normal_tostrike=155-90   #direction of the section to plot
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]

In [ ]:
utmx_close_off,utmy_close_off=utm33(np.array(close_off['lon']),np.array(close_off['lat'])) 

In [ ]:
#distance reference point
dist_ref=distance_point_from_plane(utmx_close_off/1000,utmy_close_off/1000, -close_off['depth'], normal_ref, mainev_ref_off)

# X on cross section 
X_off=+(utmy_close_off/1000-mainev_ref_off[1])*normal_ref_vertical[0]-(utmx_close_off/1000-mainev_ref_off[0])*normal_ref_vertical[1]


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 7))
#ax.scatter(X_onsection_more,(-am['depth'].loc[resultdist_more])/np.sin(dip_angle*np.pi/180),c='k',s=10,marker='.',alpha=.2,label='x')

ax.scatter(X_off,-close_off['depth']/np.sin(dip_angle*np.pi/180),c='b',s=20,marker='.',alpha=0.2)
#ax.scatter(fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180), marker='*', c='c',s=250, label='Mw 6.5')


ax.set_aspect('equal', adjustable='box')

In [ ]:
%matplotlib inline 
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.scatter(X_onsection1m,-am_poM1['depth'].loc[resultdist1m],c='gray',s=0.2,marker='.',alpha=.5,label='x')
ax.scatter(X_off,-close_off['depth'],c='r',s=0.5,marker='.',alpha=.5,label='x')
ax.scatter(X_on,-close_on['depth'],c='k',s=0.5,marker='.',alpha=0.5)
ax.scatter(fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180), marker='*', c='yellow',s=250, label='Mw 6.5')
rect = patches.Rectangle((fm_onsection_ref, -depth_ref/np.sin(dip_angle*np.pi/180)), -3, 9, angle=90,linewidth=2, edgecolor='r', facecolor='none')
ax.set_ylim(-15,0)
ax.set_xlim(-10,5)
ax.set_aspect('equal', adjustable='box')
plt.xlabel('x (km)',fontsize=22)
plt.ylabel('depth (km)',fontsize=22)
plt.savefig('./figure/2D_selected_norcia.pdf', bbox_inches='tight')

In [ ]:
%matplotlib inline
fig=plt.figure(figsize=(14,5))
plt.plot(am['time'],am['Mw'],'kx')
plt.plot(offfault_eve['time'],offfault_eve['Mw'],'o')
plt.plot(close_off['time'],close_off['Mw'],'ro')

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,figsize=(14,10))

axs[0].plot(close_off['time'],close_off['Mw'],'r.')
axs[1].plot(close_on['time'],close_on['Mw'],'k.')
axs[0].set_ylim(0,6.6)
axs[1].set_ylim(0,6.5)

In [ ]:
#catalogue to use
df = close_off.copy()
df

In [ ]:
# Reload class (e.g., to consider modifications in the code)
import importlib
import mc_lilliefors
importlib.reload(mc_lilliefors)
from mc_lilliefors import McLilliefors

lill = McLilliefors(
    df.Mw,
    # signif_lev=0.1  # [default: 0.1]
)

lill.calc_testdistr_mcutoff(
    n_repeats=50,  # number of iterations for the random noise
    Mstart=1.0,  # lowest magnitude for which to perform the test
    # log=False,  # whether to show anythe progress bar
)



In [ ]:
fig = lill.plot_testdist_expon_mcutoff()
fig.layout.update(
    xaxis_range=(1, 6.0),
)

fig.show(width=fig.layout.width, height=fig.layout.height)

In [ ]:
Mc = lill.estimate_Mc_expon_test()

print("Mc-Lilliefors: %s\n   --> number of events ≥ Mc-Lilliefors: %d" % (Mc, lill.estimates['n_compl']))

In [ ]:
Mc_off=Mc
#Mc=1.

In [ ]:
#catalogue=close_on.copy()
#magnitudes=catalogue['Mw']
catalogue=close_off.copy()
magnitudes=close_off['Mw']

In [ ]:
# Magnitude bins

min_mag=np.min(catalogue['Mw']) # estremi per il plot
max_mag=np.max(catalogue['Mw'])+interval #estremi per il plot
min_mag_bin=Mc
max_mag_bin=np.max(catalogue['Mw'])+interval

bins_off = np.arange(min_mag_bin, max_mag_bin, interval)
# Magnitude bins for plotting - we will re-arrange bins later
plot_bins_off = np.arange(min_mag, max_mag, interval)


wwww=np.where(catalogue['Mw']>min_mag_bin)
plt.hist(catalogue['Mw'].loc[wwww], bins_off, alpha = 0.5, label='a')
plt.hist(catalogue['Mw'], plot_bins_off, alpha = 0.5, label='b')
###########################################################################
# Generate distribution
###########################################################################
# Generate histogram
hist = np.histogram(catalogue['Mw'],bins=bins_off)
hist_plot = np.histogram(catalogue['Mw'],bins=plot_bins_off)

# Reverse array order
hist = hist[0][::-1]
bins_off = bins_off[::-1]
#plot 
hist_plot = hist_plot[0][::-1]
plot_bins_off = plot_bins_off[::-1]

# Calculate cumulative sum
cum_hist = hist.cumsum()
# Ensure bins have the same length has the cumulative histogram.
# Remove the upper bound for the highest interval.
bins_off = bins_off[1:]


cum_hist_plot_off = hist_plot.cumsum()
plot_bins_off = plot_bins_off[1:]
wherecum=np.where(cum_hist>0)
#log_cum_sum = np.log10(cum_hist[2::])
log_cum_sum_off = np.log10(cum_hist[wherecum])




In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
offfault_eve['Mw'][where_off].plot.hist(ax=ax, bins=50)#, bottom=1)

In [ ]:
## Compute b-value and confidence interval
#b, conf_int = b_value(mags, mt, perc=[2.5, 97.5], n_reps=10000)

mags=magnitudes
mt=Mc
compute_b_value_mle, conf_int = compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=10000)
# Generate samples to for theoretical ECDF
m_theor = np.random.exponential(compute_b_value_mle/np.log(10), size=100000) + mt 
# Plot the theoretical CDF
_ = plt.plot(*dcst.ecdf(m_theor))

# Plot the ECDF (slicing mags >= mt)
_ = plt.plot(*dcst.ecdf(mags[mags >= mt]), marker='.', linestyle='none')

# Pretty up and show the plot
_ = plt.xlabel('magnitude')
_ = plt.ylabel('ECDF')
#_ = plt.xlim(2.8, 6.2)
plt.show()

# Report the results
print("""
b-value: {0:.5f}
95% conf int: [{1:.5f}, {2:.5f}]""".format(1/compute_b_value_mle, *conf_int**(-1)))
compute_b_value_mle=1/compute_b_value_mle

In [ ]:
#b_value_off=0.99
b_off=compute_b_value_mle

In [ ]:
num_eq=len(magnitudes)
annual_num_eq = num_eq
## Get annual rate
cum_annual_rate = cum_hist

new_cum_annual_rate = []
for i in cum_annual_rate:
    new_cum_annual_rate.append(i+1e-20)

In [ ]:
# Generate data to plot maximum likelihood linear curve 

num_eve=len(catalogue['Mw'].loc[np.where(catalogue['Mw']>min_mag_bin)])
mle_fit = -1.0 * compute_b_value_mle *bins_off + 1.0 * compute_b_value_mle * min_mag_bin + np.log10(num_eve)
log_mle_fit = []
for value in mle_fit:
    log_mle_fit.append(np.power(10,value))
    
# Compare b-value of 1
fit_data = -1.0 * bins_off + min_mag_bin + np.log10(num_eve)
log_fit_data = []
for value in fit_data:
    log_fit_data.append(np.power(10,value))

print(log_cum_sum_off[-1],np.log10(num_eve))
print('compute_b_value_mle=',compute_b_value_mle)

In [ ]:
log_mle_fit_off=log_mle_fit

In [ ]:
wherecumplot_off=np.where(cum_hist_plot_off>0)
%matplotlib inline
# Plotting
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(plot_bins_off[wherecumplot_off],cum_hist_plot_off[wherecumplot_off],'xr',label = 'DhWS events of selected period')
ax.scatter(bins_off, new_cum_annual_rate, label = 'DhWS events with M>M_lilliefors')
ax.plot(bins_off, log_mle_fit, c = 'b', label = 'bvalue_MLE= %f'%b_off)
ax.plot(bins_off, log_fit_data, c = 'k', linestyle='--', label = 'b=1')


#ax.plot(bins, ls_fit2, c = 'k')
ax.set_yscale('log')
ax.legend(fontsize=24)
#ax.set_ylim([min(new_cum_annual_rate) * 0.1, max(new_cum_annual_rate) * 10.])
ax.set_ylim(1e-2, max(new_cum_annual_rate) * 100.)
ax.set_xlim([min_mag - 1, max_mag + 0.5])
ax.set_ylabel('Number events',fontsize=20)
ax.set_xlabel('Magnitude',fontsize=20)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
plt.show()



In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.plot(plot_bins_all[2::],cum_hist_plot_all[2::],'ob',label = 'Catalogue - all events in the selected time period')
#ax.scatter(bins_all, new_cum_annual_rate_all, label = 'Catalogue - all events')
ax.plot(plot_bins_off[wherecumplot_off],(cum_hist_plot_off[wherecumplot_off]),'or',label='DhWS fault - TE')
ax.plot(plot_bins[wherecumplot],(cum_hist_plot[wherecumplot]),'ok',label='On fault')
ax.plot(bins_off, log_mle_fit_off, c = 'r', label = 'bvalue_MLE_DhWS b=%.2f'%b_off)
ax.plot(bins_all, log_mle_fit_all, c = 'b', label = 'bvalue_MLE_all b=%.2f'%b_all)
ax.plot(bin_on, log_mle_fit_on, c = 'k', label = 'bvalue_MLE_on b=%.2f'%b_on)
ax.plot([Mc_on ,Mc_on], [1, 1e6], linewidth=3, color='k')
ax.plot([Mc_off ,Mc_off], [1, 1e6], linewidth=3,linestyle='--', color='r')
ax.plot([Mc_all ,Mc_all], [1, 1e6], linewidth=3, color='b')
ax.set_yscale('log')
ax.set_ylim(0.5, 1e6)
ax.legend(fontsize=20)
ax.set_ylabel('Number of events', fontsize=24)
ax.set_xlabel('Magnitude', fontsize=24)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.grid(color='0.95')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.plot(plot_bins_off[wherecumplot_off],(cum_hist_plot_off[wherecumplot_off]),'or')#,label='Off fault - TE')
ax.plot(plot_bins[wherecumplot],(cum_hist_plot[wherecumplot]),'ok')#,label='On fault')
ax.plot(bins_off, log_mle_fit_off, c = 'r', label = 'b DHwS=%.2f'%b_off)
ax.plot(bin_on, log_mle_fit_on, c = 'k', label = 'b (on fault)=%.2f'%b_on)
ax.plot([Mc_on ,Mc_on], [1, 1e6], 'k--')
ax.plot([Mc_off ,Mc_off], [1, 1e6], 'r.-')
ax.set_yscale('log')
ax.legend(fontsize=24)
ax.set_ylabel('Number of events', fontsize=24)
ax.set_xlabel('Magnitude', fontsize=24)
#ax.set_title('WHOLE PERIOD HANG & FOOT ')
ax.set_ylim(0.5, 10e4)
ax.set_xlim(0., 6.5)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.grid(color='0.95')
fig.savefig("./figure/on_fault_vs_DhWS.pdf", bbox_inches='tight')

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(20, 250)
#ww=np.where(am_poM1['depth']<12)
#ax.scatter(utmx_am_poM1[ww]/1000,utmy_am_poM1[ww]/1000,-am_poM1['depth'].loc[ww],c='grey',marker='.',alpha=0.4)
ax.set_xlabel('x(km)', fontsize='22')
ax.set_ylabel('y(km)', fontsize='22')
ax.set_zlabel('depth (km)', fontsize='22')
ax.scatter(utmx_onf[where_onf]/1000, utmy_onf[where_onf]/1000,-onfault_eve['depth'].loc[where_onf], c='k',marker='.',s=0.5)
ax.scatter(utmx_off[where_off]/1000, utmy_off[where_off]/1000,-offfault_eve['depth'].loc[where_off], c='r',marker='.',s=0.5)
plt.savefig('./figure/3D_norcia.pdf', bbox_inches='tight')


### Maps

In [ ]:
utmx_largeeve,utmy_largeeve=utm33(np.array(largeeve['lon']),np.array(largeeve['lat']))

%matplotlib inline
#%matplotlib inline
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(utmx_am/1000,utmy_am/1000, s=0.01,marker='o', c='gray', alpha=.5, label='Tan et al., 2021')
ax.scatter(utmx_off[where_off]/1000, utmy_off[where_off]/1000, c='r',marker='o',s=0.1,alpha=0.9, label='Selected')
ax.scatter(utmx_onf[where_onf]/1000, utmy_onf[where_onf]/1000, c='k',marker='o',s=0.1,alpha=.9, label='Selected')
ax.scatter(utmx_largeeve/1000, utmy_largeeve/1000, c='yellow',marker='*',s=100,alpha=.9, label='Selected')
ax.scatter(mainev_ref[0], mainev_ref[1],s=100, c='yellow',marker='*', label='Norcia Mainshock')
ax.set_xlim(330,370)
ax.set_ylim(4730,4760)

#plt.legend(fontsize=11, markerscale=2)
plt.xlabel('x (km)',fontsize=16)
plt.ylabel('y (km)',fontsize=16)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
ax=plt.gca()
ax.set_aspect('equal')
ax.grid()
plt.savefig('./figure/map_on_DhWS.pdf', bbox_inches='tight')

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(20, 250)
#ww=np.where(am_poM1['depth']<12)
#ax.scatter(utmx_am_poM1[ww]/1000,utmy_am_poM1[ww]/1000,-am_poM1['depth'].loc[ww],c='grey',marker='.',alpha=0.4)

ax.scatter(close_on['lon'], close_on['lat'],-close_on['depth'], c='k',marker='.',s=0.1)
ax.scatter(close_off['lon'], close_off['lat'],-close_off['depth'],  c='r',marker='.',s=0.1)
plt.savefig('./figure/3D_norcia.png', bbox_inches='tight')

In [ ]:

file_name_save='off_fault_wholeperiod.csv'  
close_off.to_pickle(file_name_save)

In [ ]:
file_name_save='on_fault_wholeperiod.csv'  
close_on.to_pickle(file_name_save)

## Calculate daily events:

In [ ]:
#------------my modules-----------------------
import seis_utils 
import scipy.stats

In [ ]:
%matplotlib inline
datab=close_on[((close_on['Mw'] >=1))]
datab= datab.reset_index(drop=True)

year=pd.to_datetime(datab['time']).dt.year
month=pd.to_datetime(datab['time']).dt.month
day=pd.to_datetime(datab['time']).dt.day
hour=pd.to_datetime(datab['time']).dt.hour
minutes=pd.to_datetime(datab['time']).dt.minute
seconds=pd.to_datetime(datab['time']).dt.second
a_t_decYr = np.zeros( len(datab['Mw']))
for i in range(  len(datab['Mw'])):
    a_t_decYr[i] = seis_utils.dateTime2decYr( [year[i], month[i],day[i],hour[i],minutes[i],seconds[i]])
fig = plt.figure(figsize=(12,5))
fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(18, 10))
binwidth=1/365.
ax[0].hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='k')
#plt.hist(a_t_decYr,density=False, bins=100,color='r')

#plt.hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='r')
seismic_rate_on=a_t_decYr
ax[1].scatter(datab['time'],datab['Mw'],color='k',marker='o')
ax[1].set_xlabel('time',fontsize=20)
ax[1].set_ylabel('Magnitude',fontsize=20)
ax[0].set_ylabel('Seismicity rate',fontsize=20)
ax[0].set_title('Min mag 1 ',fontsize=20)
ax[0].set(xticklabels=[])
ax[0].yaxis.set_tick_params(labelsize=16)
ax[1].xaxis.set_tick_params(labelsize=16)
ax[1].yaxis.set_tick_params(labelsize=16)
fig.savefig("./figure/onfault_magnitude_distribution.pdf", bbox_inches='tight')

In [ ]:
%matplotlib inline
databb=close_off[((close_off['Mw'] >= 1.))]
databb= databb.reset_index(drop=True)

year=pd.to_datetime(databb['time']).dt.year
month=pd.to_datetime(databb['time']).dt.month
day=pd.to_datetime(databb['time']).dt.day
hour=pd.to_datetime(databb['time']).dt.hour
minutes=pd.to_datetime(databb['time']).dt.minute
seconds=pd.to_datetime(databb['time']).dt.second
a_t_decYr = np.zeros( len(databb['Mw']))
for i in range(  len(databb['Mw'])):
    a_t_decYr[i] = seis_utils.dateTime2decYr( [year[i], month[i],day[i],hour[i],minutes[i],seconds[i]])
fig = plt.figure(figsize=(12,5))
fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(18, 10))
binwidth=1/365.
ax[0].hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='r')
seismic_rate_off=a_t_decYr
ax[1].scatter(databb['time'],databb['Mw'],color='r',marker='o')
ax[1].set_xlabel('time',fontsize=20)
ax[1].set_ylabel('Magnitude',fontsize=20)
ax[0].set_ylabel('Seismicity rate',fontsize=20)
ax[0].set_title('Min mag 1',fontsize=20)
ax[0].set(xticklabels=[])
ax[0].yaxis.set_tick_params(labelsize=16)
ax[1].xaxis.set_tick_params(labelsize=16)
ax[1].yaxis.set_tick_params(labelsize=16)
fig.savefig("./figure/DhWs_magnitude_distribution.pdf", bbox_inches='tight')

In [ ]:
%matplotlib inline
binwidth=1/365.

fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
binwidth=1/365.
ax[0].hist(seismic_rate_off, bins=np.arange(min(seismic_rate_off),max(seismic_rate_off)+binwidth,binwidth),color='r')
ax[0].hist(seismic_rate_on, bins=np.arange(min(seismic_rate_on),max(seismic_rate_on)+binwidth,binwidth),color='k',alpha=0.5)


ax[1].scatter(databb['time'],databb['Mw'],color='r',marker='o')
ax[1].scatter(datab['time'],datab['Mw'],color='k',marker='o',alpha=0.5)

ax[1].set_xlabel('time',fontsize=20)
ax[1].set_ylabel('Magnitude',fontsize=20)
ax[0].set_ylabel('Seismicity rate',fontsize=20)
ax[0].set(xticklabels=[])
ax[0].yaxis.set_tick_params(labelsize=16)
ax[1].xaxis.set_tick_params(labelsize=16,rotation=45)
ax[1].yaxis.set_tick_params(labelsize=16)
fig.savefig("./figure/DhWs_ON_FAULT.pdf", bbox_inches='tight')

## Compute b-value on Clusters C1 - C2 -C3

In [ ]:
#SELECT PERIOD:
#BEFORE VISSO-NORCIA =1
#WHOLE PERIOD =0
# FOR THE CLUSTERS WE HAVE TO SELECT PERIOD=0
PERIOD=0

In [ ]:
#EVENTS FOR THE WHOLE PERIOD OR BEFORE NORCIA ACCOUNTING FOR STAI from Herrmann et al.
start_date0 = '2016-08-15 00:01 '
end_date0 = '2016-08-24 01:36:34'#'2016-12-09 06:39' STAI 19,2 ore
start_date1 = '2016-08-24 20:36 '
end_date1 = '2016-10-26 19:18:10 '
start_date2 = '2016-10-27 09:18:10 ' #STAI 14,4 hours
end_date2 = '2016-10-30 06:40:19'
start_date3 = '2016-11-01 06:40 ' #STAI 48 hours
end_date3 = '2017-01-18 09:26'
start_date4 = '2017-01-18 21:00 ' # 9,6 hours
end_date4 = '2017-08-15 22:16'

if PERIOD==0:
    am_poM1 = am[((am['time'] >= start_date0) & (am["time"] <= end_date0)) |((am['time'] >= start_date1) & (am["time"] <= end_date1)) | ((am['time'] >= start_date2) & (am["time"] <= end_date2)) | ((am['time'] >= start_date3) & (am["time"] <= end_date3)) | ((am['time'] >= start_date4) & (am["time"] <= end_date4))]
    am_poM1= am_poM1.reset_index(drop=True)
elif PERIOD==1:
    am_poM1 = am[((am['time'] >= start_date0) & (am["time"] <= end_date0)) |((am['time'] >= start_date1) & (am["time"] <= end_date1)) ]
    am_poM1= am_poM1.reset_index(drop=True)
am_poM1

In [ ]:
#We select a reference earthquake that is at the center of the cluster C1
pern_lon=13.2366
pern_lat=42.8764
pern_depth=7.
utmx_pern,utmy_pern=utm33(pern_lon,pern_lat)
mainev_pern=[utmx_pern/1000,utmy_pern/1000, -pern_depth ]
mainev_pern=np.array(mainev_pern)

### Where CLUSTER 1

In [ ]:
#where_onf = (X_onsection_onf> -10) &(X_onsection_onf< 10)&(onfault_eve['depth']<np.abs(mainev_ref[2]))&(onfault_eve['depth']>+2)
where_cluster1 = (am_poM1['lon']>13.23) &(am_poM1['lon']<13.241)&(am_poM1['lat']>42.86)&(am_poM1['lat']<42.884)&(am_poM1['depth']>6.2)&(am_poM1['depth']<8.30)


In [ ]:
perno_cluster1=am_poM1.copy()
perno_cluster1=perno_cluster1.loc[where_cluster1]
perno_cluster1= perno_cluster1.reset_index(drop=True)

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,5))
plt.plot(am_poM1['time'],am_poM1['Mw'],'o')
plt.plot(perno_cluster1['time'],perno_cluster1['Mw'],'bo')

In [ ]:
perno_cluster1 = perno_cluster1[((perno_cluster1['time'] >= end_date2)&(perno_cluster1["time"] <= end_date3))]
perno_cluster1= perno_cluster1.reset_index(drop=True)
perno_cluster1

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,5))
plt.plot(am_poM1['time'],am_poM1['Mw'],'o')
plt.plot(perno_cluster1['time'],perno_cluster1['Mw'],'bo')

In [ ]:
#catalogue to use
df = perno_cluster1.copy()
df

In [ ]:
# Reload class (e.g., to consider modifications in the code)
import importlib
import mc_lilliefors
importlib.reload(mc_lilliefors)
from mc_lilliefors import McLilliefors

lill = McLilliefors(
    df.Mw,
    # signif_lev=0.1  # [default: 0.1]
)

lill.calc_testdistr_mcutoff(
    n_repeats=50,  # number of iterations for the random noise
    Mstart=1.0,  # lowest magnitude for which to perform the test
    # log=False,  # whether to show anythe progress bar
)



In [ ]:
fig = lill.plot_testdist_expon_mcutoff()
fig.layout.update(
    xaxis_range=(1, 6.0),
)

fig.show(width=fig.layout.width, height=fig.layout.height)

In [ ]:
Mc = lill.estimate_Mc_expon_test()

print("Mc-Lilliefors: %s\n   --> number of events ≥ Mc-Lilliefors: %d" % (Mc, lill.estimates['n_compl']))

In [ ]:
Mc_off=Mc

In [ ]:
catalogue=perno_cluster1.copy()
magnitudes=perno_cluster1['Mw']

In [ ]:
# Magnitude bins

min_mag=np.min(catalogue['Mw']) # estremi per il plot
max_mag=np.max(catalogue['Mw'])+interval #estremi per il plot
min_mag_bin=Mc
max_mag_bin=np.max(catalogue['Mw'])+interval

bins_off = np.arange(min_mag_bin, max_mag_bin, interval)
# Magnitude bins for plotting - we will re-arrange bins later
plot_bins_off = np.arange(min_mag, max_mag, interval)


wwww=np.where(catalogue['Mw']>min_mag_bin)
plt.hist(catalogue['Mw'].loc[wwww], bins_off, alpha = 0.5, label='a')
plt.hist(catalogue['Mw'], plot_bins_off, alpha = 0.5, label='b')
###########################################################################
# Generate distribution
###########################################################################
# Generate histogram
hist = np.histogram(catalogue['Mw'],bins=bins_off)
hist_plot = np.histogram(catalogue['Mw'],bins=plot_bins_off)

# Reverse array order
hist = hist[0][::-1]
bins_off = bins_off[::-1]
#plot 
hist_plot = hist_plot[0][::-1]
plot_bins_off = plot_bins_off[::-1]

# Calculate cumulative sum
cum_hist = hist.cumsum()
# Ensure bins have the same length has the cumulative histogram.
# Remove the upper bound for the highest interval.
bins_off = bins_off[1:]


cum_hist_plot_off = hist_plot.cumsum()
plot_bins_off = plot_bins_off[1:]
wherecum=np.where(cum_hist>0)
#log_cum_sum = np.log10(cum_hist[2::])
log_cum_sum_off = np.log10(cum_hist[wherecum])
wherecumplot_off=np.where(cum_hist_plot_off>0)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
perno_cluster1['Mw'].plot.hist(ax=ax, bins=50)#, bottom=1)

In [ ]:
## Compute b-value and confidence interval
#b, conf_int = b_value(mags, mt, perc=[2.5, 97.5], n_reps=10000)

mags=magnitudes
mt=Mc
compute_b_value_mle, conf_int = compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=10000)
# Generate samples to for theoretical ECDF
m_theor = np.random.exponential(compute_b_value_mle/np.log(10), size=100000) + mt 
# Plot the theoretical CDF
_ = plt.plot(*dcst.ecdf(m_theor))

# Plot the ECDF (slicing mags >= mt)
_ = plt.plot(*dcst.ecdf(mags[mags >= mt]), marker='.', linestyle='none')

# Pretty up and show the plot
_ = plt.xlabel('magnitude')
_ = plt.ylabel('ECDF')
#_ = plt.xlim(2.8, 6.2)
plt.show()

# Report the results
print("""
b-value: {0:.5f}
95% conf int: [{1:.5f}, {2:.5f}]""".format(1/compute_b_value_mle, *conf_int**(-1)))
compute_b_value_mle=1/compute_b_value_mle

In [ ]:
b_off=compute_b_value_mle

In [ ]:
num_eq=len(magnitudes)
annual_num_eq = num_eq
## Get annual rate
cum_annual_rate = cum_hist

new_cum_annual_rate = []
for i in cum_annual_rate:
    new_cum_annual_rate.append(i+1e-20)

In [ ]:
# Generate data to plot maximum likelihood linear curve Eli

num_eve=len(catalogue['Mw'].loc[np.where(catalogue['Mw']>min_mag_bin)])
mle_fit = -1.0 * compute_b_value_mle *bins_off + 1.0 * compute_b_value_mle * min_mag_bin + np.log10(num_eve)
log_mle_fit = []
for value in mle_fit:
    log_mle_fit.append(np.power(10,value))
    
# Compare b-value of 1
fit_data = -1.0 * bins_off + min_mag_bin + np.log10(num_eve)
log_fit_data = []
for value in fit_data:
    log_fit_data.append(np.power(10,value))

print('compute_b_value_mle=',compute_b_value_mle)

In [ ]:
log_mle_fit_off=log_mle_fit

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.plot(plot_bins_off[wherecumplot_off],(cum_hist_plot_off[wherecumplot_off]),'or',label='C1 - TE')
ax.plot(bins_off, log_mle_fit_off, c = 'r', label = 'bvalue_MLE_off b=%.2f'%b_off)
ax.plot([Mc_off ,Mc_off], [1, 1e6], linewidth=3, color='r')
ax.set_yscale('log')
ax.set_ylim(0.5, 1e6)
ax.legend(fontsize=24)
ax.set_ylabel('Number of events', fontsize=24)
ax.set_xlabel('Magnitude', fontsize=24)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.grid(color='0.95')

In [ ]:
clus1_plot_bins_off=plot_bins_off[wherecumplot_off]
clus1_plot_hist=(cum_hist_plot_off[wherecumplot_off])
clus1_bins_off=bins_off
clus1_log_mle_fit_off=log_mle_fit_off
clus1_Mc_off=Mc_off
b1_off=b_off

### Where CLUSTER 2

In [ ]:
where_cluster2 = (am_poM1['lon']>13.2335) &(am_poM1['lon']<13.245)&(am_poM1['lat']>42.89)&(am_poM1['lat']<42.905)&(am_poM1['depth']>2.1)&(am_poM1['depth']<4.55)
perno_cluster2=am_poM1.copy()
perno_cluster2=perno_cluster2.loc[where_cluster2]
perno_cluster2= perno_cluster2.reset_index(drop=True)

In [ ]:
perno_cluster2 = perno_cluster2[(perno_cluster2['time'] <= end_date1)]
perno_cluster2= perno_cluster2.reset_index(drop=True)

In [ ]:
where_cluster2b = (am_poM1['lon']>13.225) &(am_poM1['lon']<13.233)&(am_poM1['lat']>42.87)&(am_poM1['lat']<42.88)&(am_poM1['depth']>3)&(am_poM1['depth']<4.55)
perno_cluster2b=am_poM1.copy()
perno_cluster2b=perno_cluster2b.loc[where_cluster2b]
perno_cluster2b= perno_cluster2b.reset_index(drop=True)

In [ ]:
perno_cluster2b = perno_cluster2b[(perno_cluster2b['time'] <= end_date1)]
perno_cluster2b= perno_cluster2b.reset_index(drop=True)
perno_cluster2b

In [ ]:
where_cluster2c = (am_poM1['lon']>13.2328) &(am_poM1['lon']<13.245)&(am_poM1['lat']>42.86)&(am_poM1['lat']<42.873)&(am_poM1['depth']>2.8)&(am_poM1['depth']<4.55)
perno_cluster2c=am_poM1.copy()
perno_cluster2c=perno_cluster2c.loc[where_cluster2c]
perno_cluster2c= perno_cluster2c.reset_index(drop=True)

In [ ]:
perno_cluster2c = perno_cluster2c[(perno_cluster2c['time'] <= end_date1)]
perno_cluster2c= perno_cluster2c.reset_index(drop=True)
perno_cluster2c

In [ ]:
frames = [perno_cluster2, perno_cluster2b,perno_cluster2c]
perno_cluster2tot = pd.concat(frames)
perno_cluster2tot= perno_cluster2tot.reset_index(drop=True)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(0, 313)
ax.scatter(perno_cluster2tot['lon'],perno_cluster2tot['lat'],perno_cluster2tot['depth'], c='r',marker='.')


In [ ]:
%matplotlib inline
plt.figure(figsize=(10,5))
plt.plot(am_poM1['time'],am_poM1['Mw'],'o')
plt.plot(perno_cluster2tot['time'],perno_cluster2tot['Mw'],'ro')

In [ ]:
#catalogue to use
df = perno_cluster2tot.copy()
df

In [ ]:
# Reload class (e.g., to consider modifications in the code)
import importlib
import mc_lilliefors
importlib.reload(mc_lilliefors)
from mc_lilliefors import McLilliefors

lill = McLilliefors(
    df.Mw,
    # signif_lev=0.1  # [default: 0.1]
)

lill.calc_testdistr_mcutoff(
    n_repeats=50,  # number of iterations for the random noise
    Mstart=1.0,  # lowest magnitude for which to perform the test
    # log=False,  # whether to show anythe progress bar
)



In [ ]:
fig = lill.plot_testdist_expon_mcutoff()
fig.layout.update(
    xaxis_range=(1, 6.0),
)

fig.show(width=fig.layout.width, height=fig.layout.height)

In [ ]:
Mc = lill.estimate_Mc_expon_test()

print("Mc-Lilliefors: %s\n   --> number of events ≥ Mc-Lilliefors: %d" % (Mc, lill.estimates['n_compl']))

In [ ]:
Mc_off=Mc

In [ ]:
catalogue=perno_cluster2tot.copy()
magnitudes=perno_cluster2tot['Mw']

In [ ]:
# Magnitude bins

min_mag=np.min(catalogue['Mw']) # estremi per il plot
max_mag=np.max(catalogue['Mw'])+interval #estremi per il plot
min_mag_bin=Mc
max_mag_bin=np.max(catalogue['Mw'])+interval

bins_off = np.arange(min_mag_bin, max_mag_bin, interval)
# Magnitude bins for plotting - we will re-arrange bins later
plot_bins_off = np.arange(min_mag, max_mag, interval)


wwww=np.where(catalogue['Mw']>min_mag_bin)
plt.hist(catalogue['Mw'].loc[wwww], bins_off, alpha = 0.5, label='a')
plt.hist(catalogue['Mw'], plot_bins_off, alpha = 0.5, label='b')
###########################################################################
# Generate distribution
###########################################################################
# Generate histogram
hist = np.histogram(catalogue['Mw'],bins=bins_off)
hist_plot = np.histogram(catalogue['Mw'],bins=plot_bins_off)

# Reverse array order
hist = hist[0][::-1]
bins_off = bins_off[::-1]
#plot 
hist_plot = hist_plot[0][::-1]
plot_bins_off = plot_bins_off[::-1]

# Calculate cumulative sum
cum_hist = hist.cumsum()
# Ensure bins have the same length has the cumulative histogram.
# Remove the upper bound for the highest interval.
bins_off = bins_off[1:]


cum_hist_plot_off = hist_plot.cumsum()
plot_bins_off = plot_bins_off[1:]
wherecum=np.where(cum_hist>0)
#log_cum_sum = np.log10(cum_hist[2::])
log_cum_sum_off = np.log10(cum_hist[wherecum])
wherecumplot_off=np.where(cum_hist_plot_off>0)

In [ ]:
## Compute b-value and confidence interval
#b, conf_int = b_value(mags, mt, perc=[2.5, 97.5], n_reps=10000)

mags=magnitudes
mt=Mc
compute_b_value_mle, conf_int = compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=10000)
# Generate samples to for theoretical ECDF
m_theor = np.random.exponential(compute_b_value_mle/np.log(10), size=100000) + mt 
# Plot the theoretical CDF
_ = plt.plot(*dcst.ecdf(m_theor))

# Plot the ECDF (slicing mags >= mt)
_ = plt.plot(*dcst.ecdf(mags[mags >= mt]), marker='.', linestyle='none')

# Pretty up and show the plot
_ = plt.xlabel('magnitude')
_ = plt.ylabel('ECDF')
#_ = plt.xlim(2.8, 6.2)
plt.show()

# Report the results
print("""
b-value: {0:.5f}
95% conf int: [{1:.5f}, {2:.5f}]""".format(1/compute_b_value_mle, *conf_int**(-1)))
compute_b_value_mle=1/compute_b_value_mle

In [ ]:
b_off=compute_b_value_mle

In [ ]:
num_eq=len(magnitudes)
annual_num_eq = num_eq
## Get annual rate
cum_annual_rate = cum_hist

new_cum_annual_rate = []
for i in cum_annual_rate:
    new_cum_annual_rate.append(i+1e-20)

In [ ]:
# Generate data to plot maximum likelihood linear curve Eli

num_eve=len(catalogue['Mw'].loc[np.where(catalogue['Mw']>min_mag_bin)])
mle_fit = -1.0 * compute_b_value_mle *bins_off + 1.0 * compute_b_value_mle * min_mag_bin + np.log10(num_eve)
log_mle_fit = []
for value in mle_fit:
    log_mle_fit.append(np.power(10,value))
    
# Compare b-value of 1
fit_data = -1.0 * bins_off + min_mag_bin + np.log10(num_eve)
log_fit_data = []
for value in fit_data:
    log_fit_data.append(np.power(10,value))

print('compute_b_value_mle=',compute_b_value_mle)

In [ ]:
log_mle_fit_off=log_mle_fit

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.plot(plot_bins_off[wherecumplot_off],(cum_hist_plot_off[wherecumplot_off]),'or',label='C2 - TE')
ax.plot(bins_off, log_mle_fit_off, c = 'r', label = 'bvalue_MLE_off b=%.2f'%b_off)
ax.plot([Mc_off ,Mc_off], [1, 1e6], linewidth=3, color='r')
ax.set_yscale('log')
ax.set_ylim(0.5, 1e6)
ax.legend(fontsize=24)
ax.set_ylabel('Number of events', fontsize=24)
ax.set_xlabel('Magnitude', fontsize=24)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.grid(color='0.95')

In [ ]:
clus2tot_plot_bins_off=plot_bins_off[wherecumplot_off]
clus2tot_plot_hist=(cum_hist_plot_off[wherecumplot_off])
clus2tot_bins_off=bins_off
clus2tot_log_mle_fit_off=log_mle_fit_off
clus2tot_Mc_off=Mc_off
b2tot_off=b_off

### Where cluster 3


In [ ]:
where_cluster3 = (am_poM1['lon']>13.248) &(am_poM1['lon']<13.275)&(am_poM1['lat']>42.842)&(am_poM1['lat']<42.872)&(am_poM1['depth']>2.2)&(am_poM1['depth']<4.55)


In [ ]:
perno_cluster3=am_poM1.copy()
perno_cluster3=perno_cluster3.loc[where_cluster3]
perno_cluster3= perno_cluster3.reset_index(drop=True)

In [ ]:
%matplotlib inline
plt.figure(figsize=(10,5))
plt.plot(am_poM1['time'],am_poM1['Mw'],'o')
plt.plot(perno_cluster3['time'],perno_cluster3['Mw'],'go')

In [ ]:
perno_cluster3 = perno_cluster3[(perno_cluster3['time'] <= end_date2)]
perno_cluster3= perno_cluster3.reset_index(drop=True)
perno_cluster3

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(0, 313)
ax.scatter(perno_cluster3['lon'],perno_cluster3['lat'],perno_cluster3['depth'], c='k',marker='.')


In [ ]:
#catalogue to use
df = perno_cluster3.copy()
df

In [ ]:
# Reload class (e.g., to consider modifications in the code)
import importlib
import mc_lilliefors
importlib.reload(mc_lilliefors)
from mc_lilliefors import McLilliefors

lill = McLilliefors(
    df.Mw,
    # signif_lev=0.1  # [default: 0.1]
)

lill.calc_testdistr_mcutoff(
    n_repeats=50,  # number of iterations for the random noise
    Mstart=1.0,  # lowest magnitude for which to perform the test
    # log=False,  # whether to show anythe progress bar
)



In [ ]:
fig = lill.plot_testdist_expon_mcutoff()
fig.layout.update(
    xaxis_range=(1, 6.0),
)

fig.show(width=fig.layout.width, height=fig.layout.height)

In [ ]:
Mc = lill.estimate_Mc_expon_test()

print("Mc-Lilliefors: %s\n   --> number of events ≥ Mc-Lilliefors: %d" % (Mc, lill.estimates['n_compl']))

In [ ]:
Mc_off=Mc

In [ ]:
catalogue=perno_cluster3.copy()
magnitudes=perno_cluster3['Mw']

In [ ]:
# Magnitude bins

min_mag=np.min(catalogue['Mw']) # estremi per il plot
max_mag=np.max(catalogue['Mw'])+interval #estremi per il plot
min_mag_bin=Mc
max_mag_bin=np.max(catalogue['Mw'])+interval

bins_off = np.arange(min_mag_bin, max_mag_bin, interval)
# Magnitude bins for plotting - we will re-arrange bins later
plot_bins_off = np.arange(min_mag, max_mag, interval)


wwww=np.where(catalogue['Mw']>min_mag_bin)
plt.hist(catalogue['Mw'].loc[wwww], bins_off, alpha = 0.5, label='a')
plt.hist(catalogue['Mw'], plot_bins_off, alpha = 0.5, label='b')
###########################################################################
# Generate distribution
###########################################################################
# Generate histogram
hist = np.histogram(catalogue['Mw'],bins=bins_off)
hist_plot = np.histogram(catalogue['Mw'],bins=plot_bins_off)

# Reverse array order
hist = hist[0][::-1]
bins_off = bins_off[::-1]
#plot 
hist_plot = hist_plot[0][::-1]
plot_bins_off = plot_bins_off[::-1]

# Calculate cumulative sum
cum_hist = hist.cumsum()
# Ensure bins have the same length has the cumulative histogram.
# Remove the upper bound for the highest interval.
bins_off = bins_off[1:]


cum_hist_plot_off = hist_plot.cumsum()
plot_bins_off = plot_bins_off[1:]
wherecum=np.where(cum_hist>0)
#log_cum_sum = np.log10(cum_hist[2::])
log_cum_sum_off = np.log10(cum_hist[wherecum])
wherecumplot_off=np.where(cum_hist_plot_off>0)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
perno_cluster3['Mw'].plot.hist(ax=ax, bins=50)#, bottom=1)

In [ ]:
## Compute b-value and confidence interval
#b, conf_int = b_value(mags, mt, perc=[2.5, 97.5], n_reps=10000)

mags=magnitudes
mt=Mc
compute_b_value_mle, conf_int = compute_b_value(mags, mt,interval, perc=[2.5, 97.5], n_reps=10000)
# Generate samples to for theoretical ECDF
m_theor = np.random.exponential(compute_b_value_mle/np.log(10), size=100000) + mt 
# Plot the theoretical CDF
_ = plt.plot(*dcst.ecdf(m_theor))

# Plot the ECDF (slicing mags >= mt)
_ = plt.plot(*dcst.ecdf(mags[mags >= mt]), marker='.', linestyle='none')

# Pretty up and show the plot
_ = plt.xlabel('magnitude')
_ = plt.ylabel('ECDF')
#_ = plt.xlim(2.8, 6.2)
plt.show()

# Report the results
print("""
b-value: {0:.5f}
95% conf int: [{1:.5f}, {2:.5f}]""".format(1/compute_b_value_mle, *conf_int**(-1)))
compute_b_value_mle=1/compute_b_value_mle

In [ ]:

b_off=compute_b_value_mle

In [ ]:
num_eq=len(magnitudes)
annual_num_eq = num_eq
## Get annual rate
cum_annual_rate = cum_hist

new_cum_annual_rate = []
for i in cum_annual_rate:
    new_cum_annual_rate.append(i+1e-20)

In [ ]:
# Generate data to plot maximum likelihood linear curve Eli

num_eve=len(catalogue['Mw'].loc[np.where(catalogue['Mw']>min_mag_bin)])
mle_fit = -1.0 * compute_b_value_mle *bins_off + 1.0 * compute_b_value_mle * min_mag_bin + np.log10(num_eve)
log_mle_fit = []
for value in mle_fit:
    log_mle_fit.append(np.power(10,value))
    
# Compare b-value of 1
fit_data = -1.0 * bins_off + min_mag_bin + np.log10(num_eve)
log_fit_data = []
for value in fit_data:
    log_fit_data.append(np.power(10,value))

print('compute_b_value_mle=',compute_b_value_mle)

In [ ]:
log_mle_fit_off=log_mle_fit

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.plot(plot_bins_off[wherecumplot_off],(cum_hist_plot_off[wherecumplot_off]),'or',label='C3 fault - TE')
ax.plot(bins_off, log_mle_fit_off, c = 'r', label = 'bvalue_MLE_off b=%.2f'%b_off)
ax.plot([Mc_off ,Mc_off], [1, 1e6], linewidth=3, color='r')
ax.set_yscale('log')
ax.set_ylim(0.5, 1e6)
ax.legend(fontsize=24)
ax.set_ylabel('Number of events', fontsize=24)
ax.set_xlabel('Magnitude', fontsize=24)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.grid(color='0.95')

In [ ]:
clus3_plot_bins_off=plot_bins_off[wherecumplot_off]
clus3_plot_hist=(cum_hist_plot_off[wherecumplot_off])
clus3_bins_off=bins_off
clus3_log_mle_fit_off=log_mle_fit_off
clus3_Mc_off=Mc_off
b3_off=b_off

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.plot(clus1_plot_bins_off,clus1_plot_hist,'ob')#,label='Off fault - TE')
ax.plot(clus1_bins_off, clus1_log_mle_fit_off, c = 'b', label = ' b TE-C1=%.2f'%b1_off)
ax.plot([clus1_Mc_off ,clus1_Mc_off], [1, 1e6], linewidth=1, color='b')


ax.plot(clus2tot_plot_bins_off,clus2tot_plot_hist,'or')#,label='Off fault - TE')
ax.plot(clus2tot_bins_off, clus2tot_log_mle_fit_off, c = 'r', label = ' b TE-C2=%.2f'%b2tot_off)
ax.plot([clus2tot_Mc_off ,clus2tot_Mc_off], [1, 1e6], linewidth=1, color='r')

ax.plot(clus3_plot_bins_off,clus3_plot_hist,'og')#,label='Off fault - TE')
ax.plot(clus3_bins_off, clus3_log_mle_fit_off, c = 'g', label = ' b TE-C3=%.2f'%b3_off)
ax.plot([clus3_Mc_off ,clus3_Mc_off], [1, 1e6], linewidth=1, color='g')




ax.set_yscale('log')
ax.set_ylim(0.5, 10e4)
ax.set_xlim(0., 6.5)
ax.legend(fontsize=24)
ax.set_ylabel('Number of events', fontsize=24)
ax.set_xlabel('Magnitude', fontsize=24)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.grid(color='0.95')
fig.savefig("./figure/b_value_clusters.pdf", bbox_inches='tight')

In [ ]:
# to utm33
utmx_perno1,utmy_perno1=utm33(np.array(perno_cluster1['lon']),np.array(perno_cluster1['lat']))
utmx_perno2,utmy_perno2=utm33(np.array(perno_cluster2tot['lon']),np.array(perno_cluster2tot['lat']))
utmx_perno3,utmy_perno3=utm33(np.array(perno_cluster3['lon']),np.array(perno_cluster3['lat']))


In [ ]:

#Reference earthquake that is the center of the section
pern_lon=13.2366
pern_lat=42.8764
pern_depth=7.

utmx_pern,utmy_pern=utm33(pern_lon,pern_lat)
mainev_pern=[utmx_pern/1000,utmy_pern/1000, -pern_depth ]
mainev_pern=np.array(mainev_pern)
#print(mainev_ref)

dip_angle=90
normal_tostrike=180-90   #direction of the section to plot
normal_ref=[np.sin(dip_angle*np.pi/180)*np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)*np.sin(dip_angle*np.pi/180),np.cos(dip_angle*np.pi/180)]
normal_ref_vertical=[np.cos(normal_tostrike*np.pi/180),-np.sin(normal_tostrike*np.pi/180)]

#distance reference point
dist_pern_ref=distance_point_from_plane(utmx_am/1000,utmy_am/1000, -am['depth'], normal_ref, mainev_pern)

# selection distance < x
resultdist1_pern = np.where(dist_pern_ref <5)

# X on cross section 
X_on_pern=+(utmy_am[resultdist1_pern]/1000-mainev_pern[1])*normal_ref_vertical[0]-(utmx_am[resultdist1_pern]/1000-mainev_pern[0])*normal_ref_vertical[1]


# to utm33
#utmx_perno1,utmy_perno1=utm33(np.array(perno_cluster1['lon']),np.array(perno_cluster1['lat']))
#utmx_perno3,utmy_perno3=utm33(np.array(perno_cluster3['lon']),np.array(perno_cluster3['lat']))
#utmx_perno4,utmy_perno4=utm33(np.array(perno_cluster4['lon']),np.array(perno_cluster4['lat']))

X_onsec_perno1_p=+(utmy_perno1/1000-mainev_pern[1])*normal_ref_vertical[0]-(utmx_perno1/1000-mainev_pern[0])*normal_ref_vertical[1]
X_onsec_perno2_p=+(utmy_perno2/1000-mainev_pern[1])*normal_ref_vertical[0]-(utmx_perno2/1000-mainev_pern[0])*normal_ref_vertical[1]
X_onsec_perno3_p=+(utmy_perno3/1000-mainev_pern[1])*normal_ref_vertical[0]-(utmx_perno3/1000-mainev_pern[0])*normal_ref_vertical[1]


In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(18, 10))
ax.scatter(X_on_pern,-am['depth'].loc[resultdist1_pern]/np.sin(dip_angle*np.pi/180),c='gray',s=0.2,marker='o',alpha=.5,label='x')
ax.scatter(X_onsec_perno1_p,-perno_cluster1['depth']/np.sin(dip_angle*np.pi/180),c='b',s=0.1,marker='o',alpha=.5,label='x')

ax.scatter(X_onsec_perno2_p,-perno_cluster2tot['depth']/np.sin(dip_angle*np.pi/180),c='r',s=0.1,marker='o',alpha=.5,label='x')
ax.scatter(X_onsec_perno3_p,-perno_cluster3['depth']/np.sin(dip_angle*np.pi/180),c='g',s=0.1,marker='o',alpha=.5,label='x')


#ax.scatter(fm_onsection_ref, -pern_depth/np.sin(dip_angle*np.pi/180), marker='*', c='r',s=250, label='Mw 6.5')
#ax.set_title('5 km thick')
ax.set_ylim(-10,0)
ax.set_xlim(-5,5)
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel('along longitude [km]',fontsize=18)
ax.set_ylabel('depth[km]',fontsize=18)
ax.xaxis.set_tick_params(labelsize=18)
ax.yaxis.set_tick_params(labelsize=18)
fig.savefig("./figure/cluster_along_long.pdf", bbox_inches='tight')
plt.savefig('./figure/cluster_along_long.png', bbox_inches='tight')

In [ ]:
#utmx_grandi,utmy_grandi=utm33(np.array(grandi['lon']),np.array(grandi['lat']))
#pern_lon=13.2366
#pern_lat=42.8764
#pern_depth=7.
%matplotlib inline
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.scatter(am['lon'],am['lat'], s=0.1,marker='o', c='gray', alpha=.5)
ax.scatter(perno_cluster1['lon'],perno_cluster1['lat'], s=0.01,marker='o', c='b', alpha=.5)
ax.scatter(perno_cluster2tot['lon'],perno_cluster2tot['lat'], s=0.01,marker='o', c='r', alpha=.5)
ax.scatter(perno_cluster3['lon'],perno_cluster3['lat'], s=0.01,marker='o', c='g', alpha=.5)
plt.hlines(pern_lat,pern_lon-0.0612, pern_lon+0.0612, colors='k', linestyles='dashed') 
ax.scatter(lon_ref, lat_ref,s=200, c='yellow',marker='*', label='Norcia Mainshock')

ax.set_xlim(13.,13.3)
ax.set_ylim(42.775,43)
ax.set_xlabel('longitude(°)',fontsize=18)
ax.set_ylabel('latitude(°)',fontsize=18)
ax.xaxis.set_tick_params(labelsize=18)
ax.yaxis.set_tick_params(labelsize=18)
fig.savefig("./figure/cluster_along_map.pdf", bbox_inches='tight')
plt.savefig('./figure/cluster_along_map.png', bbox_inches='tight')

In [ ]:
%matplotlib inline
#datab=perno_cluster1.copy()
datab=perno_cluster1[(perno_cluster1['Mw'] >=1)]
datab= datab.reset_index(drop=True)

year=pd.to_datetime(datab['time']).dt.year
month=pd.to_datetime(datab['time']).dt.month
day=pd.to_datetime(datab['time']).dt.day
hour=pd.to_datetime(datab['time']).dt.hour
minutes=pd.to_datetime(datab['time']).dt.minute
seconds=pd.to_datetime(datab['time']).dt.second
a_t_decYr = np.zeros( len(datab['Mw']))
for i in range(  len(datab['Mw'])):
    a_t_decYr[i] = seis_utils.dateTime2decYr( [year[i], month[i],day[i],hour[i],minutes[i],seconds[i]])
fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

binwidth=1/365.
ax[0].hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='b')

seismic_rate_off=a_t_decYr
ax[1].scatter(datab['time'],datab['Mw'],color='b',marker='o')
ax[1].set_xlabel('time',fontsize=16)
ax[1].set_ylabel('Magnitude',fontsize=16)
ax[0].set_ylabel('Seismicity rate',fontsize=16)
ax[0].set(xticklabels=[])
ax[0].yaxis.set_tick_params(labelsize=16)
ax[1].xaxis.set_tick_params(labelsize=16,rotation=45)
ax[1].yaxis.set_tick_params(labelsize=16)
fig.savefig("./figure/seismicity_rateC1blue.pdf", bbox_inches='tight')

In [ ]:
datab=perno_cluster2tot[(perno_cluster2tot['Mw'] >=1)]
datab= datab.reset_index(drop=True)

year=pd.to_datetime(datab['time']).dt.year
month=pd.to_datetime(datab['time']).dt.month
day=pd.to_datetime(datab['time']).dt.day
hour=pd.to_datetime(datab['time']).dt.hour
minutes=pd.to_datetime(datab['time']).dt.minute
seconds=pd.to_datetime(datab['time']).dt.second
a_t_decYr = np.zeros( len(datab['Mw']))
for i in range(  len(datab['Mw'])):
    a_t_decYr[i] = seis_utils.dateTime2decYr( [year[i], month[i],day[i],hour[i],minutes[i],seconds[i]])
#fig = plt.figure(figsize=(12,5))
fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

#plt.hist(a_t_decYr,density=False, bins=100,color='r')
binwidth=1/365.
ax[0].hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='r')

#plt.hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='r')
seismic_rate_off=a_t_decYr
ax[1].scatter(datab['time'],datab['Mw'],color='r',marker='o')
ax[1].set_xlabel('time',fontsize=16)
ax[1].set_ylabel('Magnitude',fontsize=16)
ax[0].set_ylabel('Seismicity rate',fontsize=16)
ax[0].set(xticklabels=[])
ax[0].yaxis.set_tick_params(labelsize=16)
ax[1].xaxis.set_tick_params(labelsize=16,rotation=45)
ax[1].yaxis.set_tick_params(labelsize=16)
fig.savefig("./figure/seismicity_rateC2rosso.pdf", bbox_inches='tight')

In [ ]:
#datab=perno_cluster4.copy()
datab=perno_cluster3[(perno_cluster3['Mw'] >=1)]
datab= datab.reset_index(drop=True)

year=pd.to_datetime(datab['time']).dt.year
month=pd.to_datetime(datab['time']).dt.month
day=pd.to_datetime(datab['time']).dt.day
hour=pd.to_datetime(datab['time']).dt.hour
minutes=pd.to_datetime(datab['time']).dt.minute
seconds=pd.to_datetime(datab['time']).dt.second
a_t_decYr = np.zeros( len(datab['Mw']))
for i in range(  len(datab['Mw'])):
    a_t_decYr[i] = seis_utils.dateTime2decYr( [year[i], month[i],day[i],hour[i],minutes[i],seconds[i]])
#fig = plt.figure(figsize=(12,5))
fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

#plt.hist(a_t_decYr,density=False, bins=100,color='r')
binwidth=1/365.
ax[0].hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='g')

#plt.hist(a_t_decYr, bins=np.arange(min(a_t_decYr),max(a_t_decYr)+binwidth,binwidth),color='r')
seismic_rate_off=a_t_decYr
ax[1].scatter(datab['time'],datab['Mw'],color='g',marker='o')
ax[1].set_xlabel('time',fontsize=20)
ax[1].set_ylabel('Magnitude',fontsize=20)
ax[0].set_ylabel('Seismicity rate',fontsize=20)
ax[0].set(xticklabels=[])
ax[0].yaxis.set_tick_params(labelsize=16)
ax[1].xaxis.set_tick_params(labelsize=16,rotation=45)
ax[1].yaxis.set_tick_params(labelsize=16)
fig.savefig("./figure/seismicity_rateC3green.pdf", bbox_inches='tight')